# Feature Engineering with the Adult Income Dataset

1. Motivation & Setup

Goal: Predict whether an adult’s income is >50K or ≤50K using demographic and employment features.
This dataset has a good mix of numerical and categorical features, missing values, skewed distributions, and class imbalance. It’s a great playground for feature engineering.

Dataset Source: https://archive.ics.uci.edu/dataset/2/adult

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay

In [2]:
from sklearn import datasets
adult = datasets.fetch_openml(name="adult", version=2, as_frame=True)
X = adult.data
y = adult.target

In [3]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States


In [4]:
X.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


## Baseline Model (minimal preprocessing)

In [5]:
# Replace "?" with NaN
X = X.replace("?", np.nan)

# Select features
num_features = ["age", "education-num", "hours-per-week", "capital-gain", "capital-loss"]
cat_features = ["workclass", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]

let's train a base model only on the numerical features

In [6]:
X_num = X[num_features]
X_train, X_test, y_train, y_test = train_test_split(X_num, y, stratify=y, test_size=0.3, random_state=42)
baseline = LogisticRegression(max_iter=1000)
baseline.fit(X_train, y_train)
y_pred = baseline.predict(X_test)
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Baseline Accuracy: 0.8176482631543028
              precision    recall  f1-score   support

       <=50K       0.83      0.95      0.89     11147
        >50K       0.72      0.39      0.51      3506

    accuracy                           0.82     14653
   macro avg       0.78      0.67      0.70     14653
weighted avg       0.81      0.82      0.80     14653



## Handling Categorical Data

`SimpleImputer` Univariate imputer for completing missing values with simple strategies. Replace missing values using a descriptive statistic (e.g. mean, median, or most frequent) along each column, or using a constant value.

[Read more..](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

In [7]:
# Preprocess numeric (impute median + scale)
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

fill in the missing values with the mean
this dataset doesn't have any missing data, but just in case
this is a pipeline so 

In [ ]:
# # example usage of OneHotEncoder
# enc = OneHotEncoder(handle_unknown='ignore')
# oneHotEncodedColumn = enc.fit_transform(X["sex"].to_numpy().reshape(-1, 1))
# print(oneHotEncodedColumn[:25].toarray())

In [8]:
# Preprocess categorical (impute + one-hot)
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])


this one fills the missing values with a constant value called "Unknown" and then those will be ignored

In [9]:
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, num_features),
    ("cat", categorical_transformer, cat_features)
])

baseline = Pipeline([
    ("preproc", preprocessor),
    ("clf", LogisticRegression(max_iter=1000))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [10]:
baseline.fit(X_train, y_train)
y_pred = baseline.predict(X_test)
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Baseline Accuracy: 0.8551832389271822
              precision    recall  f1-score   support

       <=50K       0.88      0.94      0.91     11147
        >50K       0.75      0.60      0.66      3506

    accuracy                           0.86     14653
   macro avg       0.81      0.77      0.79     14653
weighted avg       0.85      0.86      0.85     14653



slightly more accurate weighted avg

## Feature Engineering

now we can add in new columns to maybe enhance the performance

we create the new column net_capital, has_capital_gain and work_hours_bin using 3 different kinds of strategies

In [11]:
#@title Feature creation:
# net capital, flags, hours bins
X_fe = X.copy()

X_fe["net_capital"] = X_fe["capital-gain"] - X_fe["capital-loss"]
X_fe["has_capital_gain"] = (X_fe["capital-gain"] > 0).astype(int)
X_fe["work_hours_bin"] = pd.cut(X_fe["hours-per-week"],
                                bins=[0,20,40,60,100],
                                labels=["part-time","full-time","long","extreme"])
X_fe.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,net_capital,has_capital_gain,work_hours_bin
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0,0,full-time
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0,0,long
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,0,0,full-time
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,7688,1,full-time
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,0,0,full-time


In [12]:
num_features_fe = num_features + ["net_capital"]
cat_features_fe = cat_features + ["has_capital_gain","work_hours_bin"]

In [13]:
# New preprocessor
numeric_transformer_fe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
   # ("log1p", FunctionTransformer(np.log1p, validate=False)),  # handle skew
    ("scaler", StandardScaler())
])

categorical_transformer_fe = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

the below uses SelectKBest (read more about it in sklearn docs)

In [14]:
preprocessor_fe = ColumnTransformer([
    ("num", numeric_transformer_fe, num_features_fe),
    ("cat", categorical_transformer_fe, cat_features_fe)
])

fe_pipeline = Pipeline([
    ("preproc", preprocessor_fe),
    ("selector", SelectKBest(mutual_info_classif, k=30)),
    ("clf", LogisticRegression(max_iter=1000))
])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_fe, y, stratify=y, test_size=0.3, random_state=42)

fe_pipeline.fit(X_train, y_train)
y_pred_fe = fe_pipeline.predict(X_test)
print("Feature Engineering Accuracy:", accuracy_score(y_test, y_pred_fe))
print(classification_report(y_test, y_pred_fe))

/home/rae/Documents/uni/25fal/dsk807-applied-machine-learning/.venv/lib/python3.13/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/home/rae/Documents/uni/25fal/dsk807-applied-machine-learning/.venv/lib/python3.13/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/home/rae/Documents/uni/25fal/dsk807-applied-machine-learning/.venv/lib/python3.13/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/home/rae/Documents/uni/25fal/dsk807-applied-machine-learning/.venv/lib/python3.13/site-packages/

Feature Engineering Accuracy: 0.8542278031802362
              precision    recall  f1-score   support

       <=50K       0.88      0.94      0.91     11147
        >50K       0.76      0.58      0.66      3506

    accuracy                           0.85     14653
   macro avg       0.82      0.76      0.78     14653
weighted avg       0.85      0.85      0.85     14653



85% so this didn't get any better

## Feature Extraction with *PCA*

> Add blockquote



In [16]:
pca_pipeline = Pipeline([
    ("preproc", preprocessor_fe),
    ("pca", PCA(n_components=20)),
    ("clf", LogisticRegression(max_iter=1000))
])

pca_pipeline.fit(X_train, y_train)
y_pred_pca = pca_pipeline.predict(X_test)
print("PCA-based Accuracy:", accuracy_score(y_test, y_pred_pca))

PCA-based Accuracy: 0.8508837780659251


we could run this RFE "but it will take forever"
2 hours for 98% accuracy

In [ ]:
# #@title Feature Extraction with *RFE*
# from sklearn.svm import SVC

# rfe_pipeline = Pipeline([
#     ("preproc", preprocessor_fe),
#     ("rfe", RFE(SVC(kernel="linear"), step=1)),
#     ("clf", LogisticRegression(max_iter=1000))
# ])

# rfe_pipeline.fit(X_train, y_train)
# y_pred_pca = pca_pipeline.predict(X_test)
# print("PCA-based Accuracy:", accuracy_score(y_test, y_pred_pca))

## Randomforest Classifier with Engineered Features

In [17]:
rf_pipeline = Pipeline([
    ("preproc", preprocessor_fe),
    ("clf", RandomForestClassifier(n_estimators=100, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pipeline.predict(X_test)))

Random Forest Accuracy: 0.8523851770968403


uh oh I hit run
I wasn't paying attention how long is this gonna take

oh 40s
also 85% accuracy

## Save / Load Models
- `pickle`
- `joblib`

**TLDR:** `joblib` is faster in saving/loading large `NumPy` arrays, whereas `pickle` is faster with large collections of Python objects. Therefore, if your model contains large `NumPy` arrays (as the majority of models does), joblib should be faster.

Tariq recommends joblib

In [ ]:
import pickle

# Save the model to disk
with open('rf_model.pkl', 'wb') as file:
  pickle.dump(rf_pipeline, file)

In [ ]:
# Load the model rf_model disk
with open('rf_model.pkl', 'rb') as file:
  rf_loaded_classifier = pickle.load(file)
  print("Pickle Loaded Random Forest Accuracy:", accuracy_score(y_test, rf_loaded_classifier.predict(X_test)))

In [ ]:
!pip install joblib

In [ ]:
import joblib

# Save the model to disk
joblib.dump(rf_pipeline, 'rf_classifier_model.joblib')

In [ ]:
rf_loaded_classifier_joblib = joblib.load('rf_classifier_model.joblib')
print("Joblib Loaded Random Forest Accuracy:", accuracy_score(y_test, rf_loaded_classifier_joblib.predict(X_test)))